In [1]:
# Instalar PuLP (só precisa fazer isso uma vez no Colab)
!pip install -q pulp

import pulp

# Parâmetros
l = 8  # número de linhas
c = l  # número de colunas

# Criar o modelo
model = pulp.LpProblem("Rainhas_Maximizadas", pulp.LpMaximize)

# Variáveis binárias x[i][j] = 1 se há uma rainha na posição (i, j)
x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(c)] for i in range(l)]

# Função objetivo
model += pulp.lpSum(((i + j) % 2 == 0 and 1 or 2) * x[i][j] for i in range(l) for j in range(c))

# Restrição: uma única rainha por coluna
for j in range(c):
    model += pulp.lpSum(x[i][j] for i in range(l)) <= 1

# Restrição: uma única rainha por linha
for i in range(l):
    model += pulp.lpSum(x[i][j] for j in range(c)) <= 1

# Restrição: no máximo uma rainha por diagonal principal (j - i constante)
for k in range(-(c - 1), c):
    model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if j - i == k) <= 1

# Restrição: no máximo uma rainha por diagonal secundária (i + j constante)
for k in range(2 * c):
    model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if i + j == k) <= 1

# Resolver o modelo
model.solve()

# Resultados
print("Status da otimização:", pulp.LpStatus[model.status])
print("Valor ótimo z =", pulp.value(model.objective))

# Mostrar o tabuleiro com rainhas
print("\nTabuleiro com Rainhas:")
for i in range(l):
    row = ""
    for j in range(c):
        row += "♛ " if pulp.value(x[i][j]) == 1 else ". "
    print(row)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.9 MB/s eta 0:00:00
Status da otimização: Optimal
Valor ótimo z = 12.0

Tabuleiro com Rainhas:
. ♛ . . . . . . 
. . . . . . ♛ . 
. . . . ♛ . . . 
♛ . . . . . . . 
. . . . . . . . 
. . . ♛ . . . . 
. . . . . ♛ . . 
. . ♛ . . . . . 


In [1]:
# Instalar PuLP (só precisa fazer isso uma vez no Colab)
!pip install -q pulp

import pulp
import copy
from IPython.display import display, HTML

# Parâmetros
l = 8  # número de linhas
c = l  # número de colunas

# Função para construir o modelo
def criar_modelo(proibidos=[]):
    model = pulp.LpProblem("Rainhas_Maximizadas", pulp.LpMaximize)

    # Variáveis binárias x[i][j] = 1 se há uma rainha na posição (i, j)
    x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(c)] for i in range(l)]

    # Função objetivo com pesos diferentes para casas pretas/brancas
    model += pulp.lpSum(((i + j) % 2 == 0 and 1 or 2) * x[i][j] for i in range(l) for j in range(c))

    # Restrição: uma rainha por linha e coluna
    for j in range(c):
        model += pulp.lpSum(x[i][j] for i in range(l)) <= 1
    for i in range(l):
        model += pulp.lpSum(x[i][j] for j in range(c)) <= 1

    # Diagonais principais
    for k in range(-(c - 1), c):
        model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if j - i == k) <= 1
    # Diagonais secundárias
    for k in range(2 * c):
        model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if i + j == k) <= 1

    # Restrições adicionais: posições proibidas
    for (i, j) in proibidos:
        model += x[i][j] == 0

    return model, x

# Função para desenhar o tabuleiro
def desenhar_tabuleiro(x):
    html = "<table style='border-collapse: collapse;'>"
    for i in range(l):
        html += "<tr>"
        for j in range(c):
            cor = "#EEE" if (i + j) % 2 == 0 else "#666"
            conteudo = "♛" if pulp.value(x[i][j]) == 1 else "&nbsp;"
            html += f"<td style='width: 40px; height: 40px; text-align: center; font-size: 24px; background: {cor}; color: {'black' if cor=='#EEE' else 'white'}'>{conteudo}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# Enumerar soluções diferentes
solucoes = []
proibidos = []
z_anterior = None
max_solucoes = 100  # limitar para evitar loops longos

for tentativa in range(max_solucoes):
    model, x = criar_modelo(proibidos)
    model.solve(pulp.PULP_CBC_CMD(msg=0))  # Resolver sem mostrar saída

    if model.status != pulp.LpStatusOptimal:
        break  # Sem mais soluções possíveis

    z = pulp.value(model.objective)
    if z_anterior is not None and z < z_anterior:
        break  # Parar se valor da função objetivo diminuiu

    z_anterior = z
    sol = [(i, j) for i in range(l) for j in range(c) if pulp.value(x[i][j]) == 1]
    solucoes.append((z, sol))

    # Proibir uma das posições da solução atual (última rainha, por exemplo)
    proibidos.append(sol[-1])

# Mostrar resultados
print(f"Total de soluções diferentes encontradas: {len(solucoes)}\n")

for idx, (z, sol) in enumerate(solucoes):
    print(f"Solução #{idx+1}: z = {z}")
    model, x = criar_modelo([])
    for (i, j) in sol:
        x[i][j].setInitialValue(1)
        x[i][j].fixValue()
    desenhar_tabuleiro(x)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.6 MB/s eta 0:00:00
Total de soluções diferentes encontradas: 14

Solução #1: z = 12.0


,♛,,,,,,
,,,,,,♛,
,,,,♛,,,
♛,,,,,,,
,,,,,,,
,,,♛,,,,
,,,,,♛,,
,,♛,,,,,


Solução #2: z = 12.0


,♛,,,,,,
,,,,♛,,,
,,,,,,,♛
♛,,,,,,,
,,♛,,,,,
,,,,,♛,,
,,,,,,,
,,,,,,♛,


Solução #3: z = 12.0


...

,,,,,♛,,
,,,♛,,,,
,♛,,,,,,
,,,,,,♛,
,,,,♛,,,
,,♛,,,,,
,,,,,,,♛
,,,,,,,


Solução #13: z = 12.0


,,,♛,,,,
♛,,,,,,,
,,,,,,,♛
,,,,,♛,,
,♛,,,,,,
,,,,,,♛,
,,,,♛,,,
,,,,,,,


Solução #14: z = 12.0


,♛,,,,,,
,,,,♛,,,
,,,,,,,♛
♛,,,,,,,
,,,♛,,,,
,,,,,♛,,
,,♛,,,,,
,,,,,,,
